In [99]:
import pandas as pd
import numpy as np 
import datetime as dt
import re
import os
import seaborn as sns
import scipy.stats as stats

from sklearn.preprocessing import  RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDRegressor





In [3]:
def info(a):
    print('it has dimensions {},shape {}, size {}'.format(a.ndim,a.shape,a.size))


In [42]:
raw = pd.read_csv(f'D:/TOPIC_RELATED VOCABULARY IELTS/Finland/aalto/cemsschool/Other Courses/programming course for DS/project//trip_data/yellow_tripdata_2016-06.csv')

In [43]:
raw.head()


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,09/06/2016 21:06,09/06/2016 21:13,2,0.79,-73.983360,40.760937,1,N,-73.977463,40.753979,2,6.0,0.5,0.5,0.00,0.0,0.3,7.30
1,2,09/06/2016 21:06,09/06/2016 21:35,1,5.22,-73.981720,40.736668,1,N,-73.981636,40.670242,1,22.0,0.5,0.5,4.00,0.0,0.3,27.30
2,2,09/06/2016 21:06,09/06/2016 21:13,1,1.26,-73.994316,40.751072,1,N,-74.004234,40.742168,1,6.5,0.5,0.5,1.56,0.0,0.3,9.36
3,2,09/06/2016 21:06,09/06/2016 21:36,1,7.39,-73.982361,40.773891,1,N,-73.929466,40.851540,1,26.0,0.5,0.5,1.00,0.0,0.3,28.30
4,2,09/06/2016 21:06,09/06/2016 21:23,1,3.10,-73.987106,40.733173,1,N,-73.985909,40.766445,1,13.5,0.5,0.5,2.96,0.0,0.3,17.76


In [44]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 19 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   VendorID               1048575 non-null  int64  
 1   tpep_pickup_datetime   1048575 non-null  object 
 2   tpep_dropoff_datetime  1048575 non-null  object 
 3   passenger_count        1048575 non-null  int64  
 4   trip_distance          1048575 non-null  float64
 5   pickup_longitude       1048575 non-null  float64
 6   pickup_latitude        1048575 non-null  float64
 7   RatecodeID             1048575 non-null  int64  
 8   store_and_fwd_flag     1048575 non-null  object 
 9   dropoff_longitude      1048575 non-null  float64
 10  dropoff_latitude       1048575 non-null  float64
 11  payment_type           1048575 non-null  int64  
 12  fare_amount            1048575 non-null  float64
 13  extra                  1048575 non-null  float64
 14  mta_tax           

In [24]:
raw.describe()

,VendorID,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,duration
count,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06
mean,1.528136e+00,1.636896e+00,3.010579e+00,-7.304573e+01,4.024021e+01,1.040175e+00,-7.311215e+01,4.027799e+01,1.333441e+00,1.365420e+01,3.558627e-01,4.973224e-01,1.895595e+00,3.316910e-01,2.996540e-01,1.703431e+01,3.787981e+04
std,4.992080e-01,1.301522e+00,3.750994e+00,8.232540e+00,4.535013e+00,4.787309e-01,7.934125e+00,4.369716e+00,4.888149e-01,1.304250e+02,4.574452e-01,3.936377e-02,2.656670e+00,1.798481e+00,1.358867e-02,1.306874e+02,3.022014e+05
min,1.000000e+00,0.000000e+00,0.000000e+00,-1.151684e+02,0.000000e+00,1.000000e+00,-1.151754e+02,0.000000e+00,1.000000e+00,-1.750000e+02,-4.500000e+00,-5.000000e-01,-2.600000e+01,-1.050000e+01,-3.000000e-01,-1.753000e+02,-2.400000e+02
25%,1.000000e+00,1.000000e+00,1.000000e+00,-7.399156e+01,4.073714e+01,1.000000e+00,-7.399071e+01,4.073568e+01,1.000000e+00,7.000000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,3.000000e-01,8.800000e+00,4.200000e+02
50%,2.000000e+00,1.000000e+00,1.700000e+00,-7.398096e+01,4.075434e+01,1.000000e+00,-7.397849e+01,4.075486e+01,1.000000e+00,1.000000e+01,5.000000e-01,5.000000e-01,1.450000e+00,0.000000e+00,3.000000e-01,1.236000e+01,7.200000e+02
75%,2.000000e+00,2.000000e+00,3.230000e+00,-7.396574e+01,4.076863e+01,1.000000e+00,-7.396120e+01,4.077002e+01,2.000000e+00,1.550000e+01,5.000000e-01,5.000000e-01,2.550000e+00,0.000000e+00,3.000000e-01,1.896000e+01,1.200000e+03
max,2.000000e+00,9.000000e+00,5.000000e+02,0.000000e+00,5.109830e+01,9.900000e+01,1.062469e+02,5.031214e+01,4.000000e+00,1.330578e+05,7.000000e+01,5.000000e-01,3.957000e+02,5.855800e+02,5.200000e-01,1.330637e+05,2.678400e+06


In [45]:
raw['tpep_pickup_datetime']=pd.to_datetime(raw['tpep_pickup_datetime'])
raw['tpep_dropoff_datetime']=pd.to_datetime(raw['tpep_dropoff_datetime'])

In [46]:
raw['duration']=(raw['tpep_dropoff_datetime']-raw['tpep_pickup_datetime']).dt.total_seconds()

### HANDLING UNREALISTIC DATA

In [47]:
# Location conditions 
# longitude
idxlo=raw['pickup_longitude']<-73 
idx1lo =raw['pickup_longitude']>-75
idxlo1=raw['dropoff_longitude']<-73 
idx1lo1=raw['dropoff_longitude']>-75

#Latitude
idxla = rawlofiltered['pickup_latitude']>40.3
idx1la = rawlofiltered['pickup_latitude']<41
idxla1 = rawlofiltered['dropoff_latitude']>40.3
idx1la1 = rawlofiltered['dropoff_latitude']<41



In [48]:
rawlolafiltered= raw[idxlo & idx1lo & idxlo1 &idx1lo1 & idxla & idx1la &idxla1 & idx1la1]

In [49]:
rawlolafiltered

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,duration
0,2,2016-09-06 21:06:00,2016-09-06 21:13:00,2,0.79,-73.983360,40.760937,1,N,-73.977463,40.753979,2,6.0,0.5,0.5,0.00,0.00,0.3,7.30,420.0
1,2,2016-09-06 21:06:00,2016-09-06 21:35:00,1,5.22,-73.981720,40.736668,1,N,-73.981636,40.670242,1,22.0,0.5,0.5,4.00,0.00,0.3,27.30,1740.0
2,2,2016-09-06 21:06:00,2016-09-06 21:13:00,1,1.26,-73.994316,40.751072,1,N,-74.004234,40.742168,1,6.5,0.5,0.5,1.56,0.00,0.3,9.36,420.0
3,2,2016-09-06 21:06:00,2016-09-06 21:36:00,1,7.39,-73.982361,40.773891,1,N,-73.929466,40.851540,1,26.0,0.5,0.5,1.00,0.00,0.3,28.30,1800.0
4,2,2016-09-06 21:06:00,2016-09-06 21:23:00,1,3.10,-73.987106,40.733173,1,N,-73.985909,40.766445,1,13.5,0.5,0.5,2.96,0.00,0.3,17.76,1020.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,2,2016-03-06 11:15:00,2016-03-06 11:22:00,1,0.42,-73.979027,40.745171,1,N,-73.985802,40.746269,1,6.0,0.0,0.5,1.36,0.00,0.3,8.16,420.0
1048571,2,2016-03-06 11:15:00,2016-03-06 11:46:00,1,5.80,-73.951935,40.790062,1,N,-73.994850,40.752895,1,25.0,0.0,0.5,5.16,0.00,0.3,30.96,1860.0
1048572,1,2016-03-06 11:06:00,2016-03-06 11:32:00,2,6.70,-73.995590,40.687832,1,N,-74.006721,40.751347,1,23.5,0.0,0.5,5.95,5.54,0.3,35.79,1560.0
1048573,2,2016-03-06 11:06:00,2016-03-06 11:24:00,1,1.80,-73.983688,40.734360,1,N,-74.002777,40.749432,1,12.0,0.0,0.5,2.56,0.00,0.3,15.36,1080.0


In [50]:
rawlolafiltered= rawlolafiltered[rawlolafiltered['duration']>0]

In [51]:
rawlolafiltered

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,duration
0,2,2016-09-06 21:06:00,2016-09-06 21:13:00,2,0.79,-73.983360,40.760937,1,N,-73.977463,40.753979,2,6.0,0.5,0.5,0.00,0.00,0.3,7.30,420.0
1,2,2016-09-06 21:06:00,2016-09-06 21:35:00,1,5.22,-73.981720,40.736668,1,N,-73.981636,40.670242,1,22.0,0.5,0.5,4.00,0.00,0.3,27.30,1740.0
2,2,2016-09-06 21:06:00,2016-09-06 21:13:00,1,1.26,-73.994316,40.751072,1,N,-74.004234,40.742168,1,6.5,0.5,0.5,1.56,0.00,0.3,9.36,420.0
3,2,2016-09-06 21:06:00,2016-09-06 21:36:00,1,7.39,-73.982361,40.773891,1,N,-73.929466,40.851540,1,26.0,0.5,0.5,1.00,0.00,0.3,28.30,1800.0
4,2,2016-09-06 21:06:00,2016-09-06 21:23:00,1,3.10,-73.987106,40.733173,1,N,-73.985909,40.766445,1,13.5,0.5,0.5,2.96,0.00,0.3,17.76,1020.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,2,2016-03-06 11:15:00,2016-03-06 11:22:00,1,0.42,-73.979027,40.745171,1,N,-73.985802,40.746269,1,6.0,0.0,0.5,1.36,0.00,0.3,8.16,420.0
1048571,2,2016-03-06 11:15:00,2016-03-06 11:46:00,1,5.80,-73.951935,40.790062,1,N,-73.994850,40.752895,1,25.0,0.0,0.5,5.16,0.00,0.3,30.96,1860.0
1048572,1,2016-03-06 11:06:00,2016-03-06 11:32:00,2,6.70,-73.995590,40.687832,1,N,-74.006721,40.751347,1,23.5,0.0,0.5,5.95,5.54,0.3,35.79,1560.0
1048573,2,2016-03-06 11:06:00,2016-03-06 11:24:00,1,1.80,-73.983688,40.734360,1,N,-74.002777,40.749432,1,12.0,0.0,0.5,2.56,0.00,0.3,15.36,1080.0


In [52]:
# Speed conditions
sc1= (rawlolafiltered['trip_distance']/rawlolafiltered['duration'])<=0.01 # < 38MPH
sc2= (rawlolafiltered['trip_distance']/rawlolafiltered['duration'])>0.00097 # >3.5MPH
rawlolafiltered1= rawlolafiltered[sc1 & sc2]

In [53]:
#Fare amount conditions
fc1= rawlolafiltered['fare_amount']<rawlolafiltered['trip_distance']*10
fc2= rawlolafiltered['fare_amount']>rawlolafiltered['trip_distance']*2.5
rawlolafiltered2= rawlolafiltered1[fc1 & fc2]

<ipython-input-53-7bc0319042de>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rawlolafiltered2= rawlolafiltered1[fc1 & fc2]


In [54]:
raw= rawlolafiltered2

In [55]:
raw

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,duration
0,2,2016-09-06 21:06:00,2016-09-06 21:13:00,2,0.79,-73.983360,40.760937,1,N,-73.977463,40.753979,2,6.0,0.5,0.5,0.00,0.00,0.3,7.30,420.0
1,2,2016-09-06 21:06:00,2016-09-06 21:35:00,1,5.22,-73.981720,40.736668,1,N,-73.981636,40.670242,1,22.0,0.5,0.5,4.00,0.00,0.3,27.30,1740.0
2,2,2016-09-06 21:06:00,2016-09-06 21:13:00,1,1.26,-73.994316,40.751072,1,N,-74.004234,40.742168,1,6.5,0.5,0.5,1.56,0.00,0.3,9.36,420.0
3,2,2016-09-06 21:06:00,2016-09-06 21:36:00,1,7.39,-73.982361,40.773891,1,N,-73.929466,40.851540,1,26.0,0.5,0.5,1.00,0.00,0.3,28.30,1800.0
4,2,2016-09-06 21:06:00,2016-09-06 21:23:00,1,3.10,-73.987106,40.733173,1,N,-73.985909,40.766445,1,13.5,0.5,0.5,2.96,0.00,0.3,17.76,1020.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048569,2,2016-03-06 11:15:00,2016-03-06 11:27:00,1,1.42,-73.958763,40.778252,1,N,-73.978683,40.783211,1,9.0,0.0,0.5,1.20,0.00,0.3,11.00,720.0
1048571,2,2016-03-06 11:15:00,2016-03-06 11:46:00,1,5.80,-73.951935,40.790062,1,N,-73.994850,40.752895,1,25.0,0.0,0.5,5.16,0.00,0.3,30.96,1860.0
1048572,1,2016-03-06 11:06:00,2016-03-06 11:32:00,2,6.70,-73.995590,40.687832,1,N,-74.006721,40.751347,1,23.5,0.0,0.5,5.95,5.54,0.3,35.79,1560.0
1048573,2,2016-03-06 11:06:00,2016-03-06 11:24:00,1,1.80,-73.983688,40.734360,1,N,-74.002777,40.749432,1,12.0,0.0,0.5,2.56,0.00,0.3,15.36,1080.0


In [56]:
# AUGMENTED DATASET
raw['start_month']=raw['tpep_pickup_datetime'].dt.month_name()
raw['end_month']=raw['tpep_dropoff_datetime'].dt.month_name()
months =['January','February','March', 'April','May','June','July','August','September','October','November','December']
dmonths= dict(zip(months,np.arange(1,13,1)))
raw['start_month_num']=raw['start_month'].map(dmonths)
raw['end_month_num']=raw['end_month'].map(dmonths)

raw['start_day']=raw['tpep_pickup_datetime'].dt.dayofweek
raw['end_day']=raw['tpep_dropoff_datetime'].dt.dayofweek
dday= dict(zip(np.arange(0,7,1),[0,0,0,0,0,1,1]))
raw['weekend_starttrip']=raw['start_day'].map(dday)
raw['weekend_endtrip']=raw['end_day'].map(dday)

raw['start_hour']=raw['tpep_pickup_datetime'].dt.hour
raw['end_hour']=raw['tpep_dropoff_datetime'].dt.hour







<ipython-input-56-2c4379d363cf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw['start_month']=raw['tpep_pickup_datetime'].dt.month_name()
<ipython-input-56-2c4379d363cf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw['end_month']=raw['tpep_dropoff_datetime'].dt.month_name()
<ipython-input-56-2c4379d363cf>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

### SIN COS TRANSFORM FOR PERIODICITY FEATURES

In [57]:

raw['start_hour_sin']= np.sin(raw['start_hour']*(2.*np.pi/24)) # coordinate values on Oy
raw['start_hour_cos']= np.cos(raw['start_hour']*(2.*np.pi/24)) #coordinate values on Ox

raw['start_day_sin']= np.sin(raw['start_day']*(2.*np.pi/7))
raw['start_day_cos']= np.cos(raw['start_day']*(2.*np.pi/7))


raw['start_month_sin']= np.sin(raw['start_month_num']*(2.*np.pi/12))
raw['start_month_cos']= np.cos(raw['start_month_num']*(2.*np.pi/12))


<ipython-input-57-e426ed6db2a0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw['start_hour_sin']= np.sin(raw['start_hour']*(2.*np.pi/24)) # coordinate values on Oy
<ipython-input-57-e426ed6db2a0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw['start_hour_cos']= np.cos(raw['start_hour']*(2.*np.pi/24)) #coordinate values on Ox
<ipython-input-57-e426ed6db2a0>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

In [58]:
raw.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,...,weekend_starttrip,weekend_endtrip,start_hour,end_hour,start_hour_sin,start_hour_cos,start_day_sin,start_day_cos,start_month_sin,start_month_cos
0,2,2016-09-06 21:06:00,2016-09-06 21:13:00,2,0.79,-73.983360,40.760937,1,N,-73.977463,...,0,0,21,21,-0.707107,0.707107,0.781831,0.62349,-1.0,-1.836970e-16
1,2,2016-09-06 21:06:00,2016-09-06 21:35:00,1,5.22,-73.981720,40.736668,1,N,-73.981636,...,0,0,21,21,-0.707107,0.707107,0.781831,0.62349,-1.0,-1.836970e-16
2,2,2016-09-06 21:06:00,2016-09-06 21:13:00,1,1.26,-73.994316,40.751072,1,N,-74.004234,...,0,0,21,21,-0.707107,0.707107,0.781831,0.62349,-1.0,-1.836970e-16
3,2,2016-09-06 21:06:00,2016-09-06 21:36:00,1,7.39,-73.982361,40.773891,1,N,-73.929466,...,0,0,21,21,-0.707107,0.707107,0.781831,0.62349,-1.0,-1.836970e-16
4,2,2016-09-06 21:06:00,2016-09-06 21:23:00,1,3.10,-73.987106,40.733173,1,N,-73.985909,...,0,0,21,21,-0.707107,0.707107,0.781831,0.62349,-1.0,-1.836970e-16


In [59]:
# BUILD SEASONS AND DAYPARTS FEATURES BASED ON TIME 
def seasons(month):
    if month in [11,12,1]:
        return 'Winter'
    elif month in [2,3,4]:
        return 'Spring'
    elif month in [5,6,7]:
        return 'Summer'
    else: 
        return 'Fall'


In [60]:
raw['seasons']=raw['start_month_num'].map(seasons)

<ipython-input-60-61ef6c99acb5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw['seasons']=raw['start_month_num'].map(seasons)


In [66]:
rawseasons=pd.get_dummies(raw['seasons'])

In [67]:
rawseasons

,Fall,Spring,Winter
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
...,...,...,...
1048569,0,1,0
1048571,0,1,0
1048572,0,1,0
1048573,0,1,0


In [62]:
hours= np.arange(0,24)
sdayparts=['smidnight','smidnight','sdawn','sdawn','sdawn','sdawn','smorning','smorning','smorning','smorning','snoon','snoon','snoon','snoon','safternoon','safternoon','safternoon','safternoon','sevening','sevening','sevening','sevening','smidnight','smidnight']
dsdayparts= dict(zip(hours,sdayparts))


In [63]:
raw['start_dayparts']=raw['start_hour'].map(dsdayparts)

<ipython-input-63-2c3e7cdc998a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw['start_dayparts']=raw['start_hour'].map(dsdayparts)


In [64]:
start_dayparts = pd.get_dummies(raw['start_dayparts'],drop_first=True)

In [65]:
start_dayparts

,sdawn,sevening,smidnight,smorning,snoon
0,0,1,0,0,0
1,0,1,0,0,0
2,0,1,0,0,0
3,0,1,0,0,0
4,0,1,0,0,0
...,...,...,...,...,...
1048569,0,0,0,0,1
1048571,0,0,0,0,1
1048572,0,0,0,0,1
1048573,0,0,0,0,1


In [68]:
raw= pd.concat((raw,start_dayparts, rawseasons),axis=1)

In [69]:
raw=raw.loc[:,~raw.columns.duplicated()]

In [70]:
raw.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,...,seasons,start_dayparts,sdawn,sevening,smidnight,smorning,snoon,Fall,Spring,Winter
0,2,2016-09-06 21:06:00,2016-09-06 21:13:00,2,0.79,-73.983360,40.760937,1,N,-73.977463,...,Fall,sevening,0,1,0,0,0,1,0,0
1,2,2016-09-06 21:06:00,2016-09-06 21:35:00,1,5.22,-73.981720,40.736668,1,N,-73.981636,...,Fall,sevening,0,1,0,0,0,1,0,0
2,2,2016-09-06 21:06:00,2016-09-06 21:13:00,1,1.26,-73.994316,40.751072,1,N,-74.004234,...,Fall,sevening,0,1,0,0,0,1,0,0
3,2,2016-09-06 21:06:00,2016-09-06 21:36:00,1,7.39,-73.982361,40.773891,1,N,-73.929466,...,Fall,sevening,0,1,0,0,0,1,0,0
4,2,2016-09-06 21:06:00,2016-09-06 21:23:00,1,3.10,-73.987106,40.733173,1,N,-73.985909,...,Fall,sevening,0,1,0,0,0,1,0,0


### LONGTITUDE LATITUDE FEATURES HANDLING

In [100]:
from math import radians

In [101]:
###LON LAT OF SOME SPECIFIC PLACES
JFKairport= (-73.780968, 40.641766)

NWairport=(-74.172363, 40.735657)

#MIDTOWN MANHATTAN
#UES the mark hotel 40.77541739176158, -73.96342029243233
Mark= (-73.96342029243233,40.77541739176158)
Timesquare = (-73.98604331729206,40.75837054036599)
Grand =( -73.97711770261203,40.75284316174533)
Penn= (-73.99368229963214, 40.75069857282476)
Lincoln= (-73.9839037407981, 40.773072825611635)

#WEST SIDE MANHATTAN Bloomingdale flor de mayo 40.797800366747595, -73.9700034852982
Flor = (-73.9700034852982,40.797800366747595)

# LOWER MANHATTAN one world trade center 40.71315120718066, -74.01335094122703
Wtc=  (-74.01335094122703,40.71315120718066)

#UPPER MANHATTAN bank of america 40.82320010657351, -73.9420433120473
Boa = (-73.9420433120473, 40.82320010657351)

L= [JFKairport, NWairport, Mark,Timesquare,Grand,Penn,Lincoln,Flor,Wtc,Boa]

In [102]:
#Converting decimal lon lat to radians for each specific places mentioned above
def detora(List):
    res=[]
    for i in List:
        listc= list(map(radians,i))
        res.append(listc)
    return res
res= detora(L)

In [74]:
res

[[-1.2877208169085756, 0.7093326305250857],
 [-1.294551948334419, 0.7109713376130758],
 [-1.2909052101393315, 0.7116652873611982],
 [-1.2913000564098942, 0.7113677636772469],
 [-1.2911442750459634, 0.7112712927213121],
 [-1.2914333818032302, 0.7112338625835463],
 [-1.2912627137554762, 0.7116243669623508],
 [-1.2910201085301345, 0.7120559439710956],
 [-1.2917766643584558, 0.7105785374276066],
 [-1.2905321114474784, 0.7124992530602078]]

In [75]:
# convert decimal lon lat to radians for PUlon and PUlat in the 'raw'  dataframe
puloc= pd.Series(map(radians,raw['pickup_longitude']))
pulac= pd.Series(map(radians,raw['pickup_latitude']))

In [77]:
puloc

0        -1.291253
1        -1.291225
2        -1.291444
3        -1.291236
4        -1.291319
            ...   
924428   -1.290824
924429   -1.290705
924430   -1.291467
924431   -1.291259
924432   -1.291053
Length: 924433, dtype: float64

In [76]:
#Compute the Harvesine distance between the datapoints and each specific place listed above
def haversineindf(df):
    for i in np.arange(0,10):
        dlon= puloc-res[i][0]
        dlat= pulac-res[i][1]
        a= np.sin(dlat/2)**2 + np.cos(pulac) * cos(res[0][1]) * np.sin(dlon/2)**2
        c= 2*np.arcsin(np.sqrt(a))
        s= pd.Series(c*3956).rename(i)
        df= pd.concat((df,s),axis=1)
    return df

In [78]:
s1= pd.Series(np.zeros(924433))
df= haversineindf(s1)

In [79]:
df=df.iloc[:,1::]

In [80]:
df.head()

,0,1,2,3,4,5,6,7,8,9
0,13.414098,10.046033,1.445355,0.226093,0.647347,0.889734,0.838421,2.639536,3.653793,4.812342
1,12.385433,9.981167,2.841849,1.515480,1.142539,1.153575,2.516204,4.265262,2.319214,6.325456
2,13.479301,9.381028,2.332671,0.664452,0.908575,0.042016,1.613888,3.468332,2.801447,5.682368
3,13.940239,10.289019,0.996919,1.088839,1.478966,1.707475,0.098570,1.772977,4.496559,4.005461
4,12.502349,9.700696,3.169423,1.740634,1.455313,1.258058,2.759965,4.551141,1.949136,6.648599


In [82]:
scale= RobustScaler() # due to outliers in  df
scale.fit(df)


RobustScaler()

In [104]:
dfscaled= scale.transform(df)

In [105]:
dfscaled= pd.DataFrame(dfscaled.tolist(),index=raw.index)

In [106]:
dfscaled

,0,1,2,3,4,5,6,7,8,9
0,0.475537,-0.077752,-0.260835,-0.824207,-0.568678,-0.381146,-0.544176,-0.247874,0.085345,-0.185434
1,-0.782766,-0.118446,0.377546,0.012130,-0.230407,-0.218393,0.472415,0.449505,-0.431171,0.454778
2,0.555295,-0.494945,0.144785,-0.539874,-0.390230,-0.904066,-0.074310,0.107650,-0.244535,0.182682
3,1.119133,0.074686,-0.465828,-0.264603,-0.000590,0.123283,-0.992462,-0.619597,0.411516,-0.526832
4,-0.639750,-0.294400,0.527290,0.158171,-0.016748,-0.153942,0.620113,0.572137,-0.574400,0.591502
...,...,...,...,...,...,...,...,...,...,...
1048569,0.267111,0.871840,-0.778660,0.313833,0.355420,0.697306,-0.226011,-0.748542,0.732532,-0.857268
1048571,0.699890,1.230801,-0.383828,0.860785,0.961943,1.221064,0.185865,-0.914243,1.073593,-1.229024
1048572,-1.643494,-0.213652,1.948171,2.204811,2.125815,1.748281,2.533136,1.927237,-0.562360,1.907200
1048573,-0.776947,-0.182887,0.462151,0.107432,-0.107991,-0.162890,0.567406,0.523810,-0.502662,0.532850


In [107]:
dfscaled=dfscaled.rename(columns=dict(zip(np.arange(0,10),['tojfk','tonw', 'tomark','totimes','togrand','topenn','tolincoln','toflor','towtc','toboa'])))

In [108]:
#the haversine distance computed represented by a dataframe with informative column names
dfscaled

,tojfk,tonw,tomark,totimes,togrand,topenn,tolincoln,toflor,towtc,toboa
0,0.475537,-0.077752,-0.260835,-0.824207,-0.568678,-0.381146,-0.544176,-0.247874,0.085345,-0.185434
1,-0.782766,-0.118446,0.377546,0.012130,-0.230407,-0.218393,0.472415,0.449505,-0.431171,0.454778
2,0.555295,-0.494945,0.144785,-0.539874,-0.390230,-0.904066,-0.074310,0.107650,-0.244535,0.182682
3,1.119133,0.074686,-0.465828,-0.264603,-0.000590,0.123283,-0.992462,-0.619597,0.411516,-0.526832
4,-0.639750,-0.294400,0.527290,0.158171,-0.016748,-0.153942,0.620113,0.572137,-0.574400,0.591502
...,...,...,...,...,...,...,...,...,...,...
1048569,0.267111,0.871840,-0.778660,0.313833,0.355420,0.697306,-0.226011,-0.748542,0.732532,-0.857268
1048571,0.699890,1.230801,-0.383828,0.860785,0.961943,1.221064,0.185865,-0.914243,1.073593,-1.229024
1048572,-1.643494,-0.213652,1.948171,2.204811,2.125815,1.748281,2.533136,1.927237,-0.562360,1.907200
1048573,-0.776947,-0.182887,0.462151,0.107432,-0.107991,-0.162890,0.567406,0.523810,-0.502662,0.532850


In [109]:
X= pd.concat((raw,dfscaled),axis=1)


### PREDICTING TRIP DISTANCE

In [110]:
ySGD= X['trip_distance']

In [111]:
X.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'pickup_longitude',
       'pickup_latitude', 'RatecodeID', 'store_and_fwd_flag',
       'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount', 'duration', 'start_month',
       'end_month', 'start_month_num', 'end_month_num', 'start_day', 'end_day',
       'weekend_starttrip', 'weekend_endtrip', 'start_hour', 'end_hour',
       'start_hour_sin', 'start_hour_cos', 'start_day_sin', 'start_day_cos',
       'start_month_sin', 'start_month_cos', 'seasons', 'start_dayparts',
       'sdawn', 'sevening', 'smidnight', 'smorning', 'snoon', 'Fall', 'Spring',
       'Winter', 'tojfk', 'tonw', 'tomark', 'totimes', 'togrand', 'topenn',
       'tolincoln', 'toflor', 'towtc', 'toboa'],
      dtype='object')

In [112]:
#Select only columns of interest
XSGD=X[['start_hour_sin',        'start_hour_cos',
               'start_day_sin',         'start_day_cos',
             'start_month_sin',       'start_month_cos',
                       'sdawn',              'sevening',
                   'smidnight',              'smorning',
                       'snoon',                  'Fall',
                      'Spring',                'Winter',
            'tojfk','tonw', 'tomark','totimes','togrand','topenn','tolincoln','toflor','towtc','toboa']]


In [113]:
XSGD

,start_hour_sin,start_hour_cos,start_day_sin,start_day_cos,start_month_sin,start_month_cos,sdawn,sevening,smidnight,smorning,...,tojfk,tonw,tomark,totimes,togrand,topenn,tolincoln,toflor,towtc,toboa
0,-0.707107,0.707107,0.781831,0.62349,-1.0,-1.836970e-16,0,1,0,0,...,0.475537,-0.077752,-0.260835,-0.824207,-0.568678,-0.381146,-0.544176,-0.247874,0.085345,-0.185434
1,-0.707107,0.707107,0.781831,0.62349,-1.0,-1.836970e-16,0,1,0,0,...,-0.782766,-0.118446,0.377546,0.012130,-0.230407,-0.218393,0.472415,0.449505,-0.431171,0.454778
2,-0.707107,0.707107,0.781831,0.62349,-1.0,-1.836970e-16,0,1,0,0,...,0.555295,-0.494945,0.144785,-0.539874,-0.390230,-0.904066,-0.074310,0.107650,-0.244535,0.182682
3,-0.707107,0.707107,0.781831,0.62349,-1.0,-1.836970e-16,0,1,0,0,...,1.119133,0.074686,-0.465828,-0.264603,-0.000590,0.123283,-0.992462,-0.619597,0.411516,-0.526832
4,-0.707107,0.707107,0.781831,0.62349,-1.0,-1.836970e-16,0,1,0,0,...,-0.639750,-0.294400,0.527290,0.158171,-0.016748,-0.153942,0.620113,0.572137,-0.574400,0.591502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048569,0.258819,-0.965926,-0.781831,0.62349,1.0,6.123234e-17,0,0,0,0,...,0.267111,0.871840,-0.778660,0.313833,0.355420,0.697306,-0.226011,-0.748542,0.732532,-0.857268
1048571,0.258819,-0.965926,-0.781831,0.62349,1.0,6.123234e-17,0,0,0,0,...,0.699890,1.230801,-0.383828,0.860785,0.961943,1.221064,0.185865,-0.914243,1.073593,-1.229024
1048572,0.258819,-0.965926,-0.781831,0.62349,1.0,6.123234e-17,0,0,0,0,...,-1.643494,-0.213652,1.948171,2.204811,2.125815,1.748281,2.533136,1.927237,-0.562360,1.907200
1048573,0.258819,-0.965926,-0.781831,0.62349,1.0,6.123234e-17,0,0,0,0,...,-0.776947,-0.182887,0.462151,0.107432,-0.107991,-0.162890,0.567406,0.523810,-0.502662,0.532850


In [114]:
Xtrain,Xtest,ytrain,ytest=train_test_split(XSGD,ySGD,test_size=0.2)

In [115]:
model = SGDRegressor()

In [116]:
model.fit(Xtrain,ytrain)

SGDRegressor()

In [117]:
print(model.score(Xtrain,ytrain))

0.35611174470093765


In [118]:
print(model.score(Xtest, ytest))

0.35542316088285275
